In [1]:
import pandas as pd
import os
import sys
from sklearn import preprocessing
from IPython.display import display
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier as KNN
import xgboost as xgb
import yake
import nltk
#nltk.download()

In [2]:
vaers_16 = pd.read_csv(os.path.join(sys.path[0]+"\\2016VAERSData\\2016VAERSDATA.csv"), encoding='latin-1')
vax_16 = pd.read_csv(os.path.join(sys.path[0]+"\\2016VAERSData\\2016VAERSVAX.csv"), encoding='latin-1')
sym_16 = pd.read_csv(os.path.join(sys.path[0]+"\\2016VAERSData\\2016VAERSSYMPTOMS.csv"), encoding='latin-1')

vaers_17 = pd.read_csv(os.path.join(sys.path[0]+"\\2017VAERSData\\2017VAERSDATA.csv"), encoding='latin-1', low_memory=False)
vax_17 = pd.read_csv(os.path.join(sys.path[0]+"\\2017VAERSData\\2017VAERSVAX.csv"), encoding='latin-1')
sym_17 = pd.read_csv(os.path.join(sys.path[0]+"\\2017VAERSData\\2017VAERSSYMPTOMS.csv"), encoding='latin-1')

## Vax

In [3]:
print(vax_16.shape,vax_17.shape)
vax = pd.concat([vax_16, vax_17], ignore_index=True)
vax.replace(["UN","UNK", "Unkown", "UNKNOWN MANUFACTURER"], np.nan, inplace=True)
print("Raw merge: ",vax.shape)

# Choose attrs, excluded lot number(batch no.)
vax.drop(columns=["VAX_NAME", "VAX_LOT"], inplace=True)
print("With selected ft.: ",vax.shape)

#Ensure VAERS_ID is unique
print ("\nVAERS_ID is unique.\n" if len(vax["VAERS_ID"])==vax.shape[0] else "\n VAERS_ID isn't unique.\n")

# Check Missing values
for col in vax.columns:
    print(col, "has %d missing values.\n %s" % ((vax[col].isna().sum()), \
                                                "{0:.1f}%".format((vax[col].isna().sum())/vax.shape[0] * 100)))
display(vax.head(3))
print(vax.shape)

(60449, 8) (52751, 8)
Raw merge:  (113200, 8)
With selected ft.:  (113200, 6)

VAERS_ID is unique.

VAERS_ID has 0 missing values.
 0.0%
VAX_TYPE has 450 missing values.
 0.4%
VAX_MANU has 7541 missing values.
 6.7%
VAX_DOSE_SERIES has 60229 missing values.
 53.2%
VAX_ROUTE has 41878 missing values.
 37.0%
VAX_SITE has 44798 missing values.
 39.6%


,VAERS_ID,VAX_TYPE,VAX_MANU,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE
0,617010,FLU3,SANOFI PASTEUR,1,IM,LA
1,617011,FLU4,GLAXOSMITHKLINE BIOLOGICALS,1,IM,RA
2,617012,PPV,NaN,NaN,NaN,NaN


(113200, 6)


### Dealing Missing Values

In [4]:
label_codes = {}
# drop rows with more than 1 missing value, can't predict
vax = vax[vax.isna().sum(axis=1) < 2]
print(vax.shape)
display(vax.head(5))

(66842, 6)


,VAERS_ID,VAX_TYPE,VAX_MANU,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE
0,617010,FLU3,SANOFI PASTEUR,1,IM,LA
1,617011,FLU4,GLAXOSMITHKLINE BIOLOGICALS,1,IM,RA
4,617014,MMR,NaN,2,SYR,LA
7,617016,PNC13,PFIZER\WYETH,2,IM,LA
8,617017,FLU3,CSL LIMITED,NaN,IM,LA


In [5]:
def Encode(df):
    mask = df.isna()
    
    le = preprocessing.LabelEncoder()
    df_coded = df.copy()
    code_columns = list(df_coded.columns)
    code_columns.remove("VAERS_ID")
    for col in df_coded.columns[1:]:
        df_coded[col] = le.fit_transform(df_coded[col])
        global label_codes
        label_codes[col] = le.classes_[:]
        print(col, len(le.classes_), le.classes_, "\n")
        
    df_coded = df_coded.where(~mask, df)
        
    return df_coded

### Fill with XGBoost & KNN

In [6]:
# Get full sub-dataset to train model
vax_coded = Encode(vax)
vax_coded["VAERS_ID"] = vax["VAERS_ID"]
vax_full = vax_coded[~vax_coded.isnull().any(axis=1)]
print("full subset: ", vax_full.shape)
vax_full = vax_full.astype("int64")
vax_full.dtypes

VAX_TYPE 75 ['6VAX-F' 'ADEN_4_7' 'ANTH' 'BCG' 'CHOL' 'DT' 'DTAP' 'DTAPH' 'DTAPHEPBIP'
 'DTAPIPV' 'DTAPIPVHIB' 'DTIPV' 'DTOX' 'DTP' 'DTPHIB' 'DTPIHI' 'DTPIPV'
 'DTPPHIB' 'FLU(H1N1)' 'FLU3' 'FLU4' 'FLUA3' 'FLUC3' 'FLUC4' 'FLUN(H1N1)'
 'FLUN3' 'FLUN4' 'FLUR3' 'FLUR4' 'FLUX' 'FLUX(H1N1)' 'HBHEPB' 'HBPV' 'HEP'
 'HEPA' 'HEPAB' 'HIBV' 'HPV2' 'HPV4' 'HPV9' 'HPVX' 'IPV' 'JEV' 'JEV1'
 'JEVX' 'LYME' 'MEA' 'MEN' 'MENB' 'MENHIB' 'MER' 'MMR' 'MMRV' 'MNQ'
 'MNQHIB' 'OPV' 'PER' 'PNC' 'PNC10' 'PNC13' 'PPV' 'RAB' 'RUB' 'RV1' 'RV5'
 'RVX' 'SMALL' 'TD' 'TDAP' 'TDAPIPV' 'TTOX' 'TYP' 'VARCEL' 'VARZOS' 'YF'] 

VAX_MANU 24 ['AVENTIS PASTEUR' 'BAXTER HEALTHCARE CORP.' 'BERNA BIOTECH, LTD'
 'BERNA BIOTECH, LTD.' 'CONNAUGHT LABORATORIES' 'CSL LIMITED'
 'EMERGENT BIOSOLUTIONS' 'GLAXOSMITHKLINE BIOLOGICALS' 'INTERCELL AG'
 'MASS. PUB HLTH BIOL LAB' 'MEDEVA PHARMA, LTD.'
 'MEDIMMUNE VACCINES, INC.' 'MERCK & CO. INC.' 'MICHIGAN DEPT PUB HLTH'
 'NOVARTIS VACCINES AND DIAGNOSTICS' 'PASTEUR MERIEUX INST.' 'PAXVAX'
 'PF

VAERS_ID           int64
VAX_TYPE           int64
VAX_MANU           int64
VAX_DOSE_SERIES    int64
VAX_ROUTE          int64
VAX_SITE           int64
dtype: object

In [7]:
def xgb_train(full_Xset, full_yset, seed):
    # Split train / test set
    train_x, test_x, train_y, test_y = train_test_split(full_Xset, full_yset, test_size=0.2, random_state=42)
    
    
    model = xgb.XGBClassifier(seed=seed, n_estimators=15, max_depth=5, learning_rate=0.1, eval_metric="merror",)
    model.fit(train_x, train_y,  verbose=10)
    
    # make predictions for test data
    y_pred = model.predict(test_x)
    predictions = [round(value) for value in y_pred]

    # evaluate predictions
    accuracy = accuracy_score(test_y, predictions)
    print ("Accuracy: %.2f%%" % (accuracy * 100.0))

    return model

def knn_train(full_Xset, full_yset, n):
    # Split train / test set
    train_x, test_x, train_y, test_y = train_test_split(full_Xset, full_yset, test_size=0.3, random_state=42)
    
    model = KNN(n_neighbors=n).fit(train_x, train_y)
    
    # make predictions for test data
    y_pred = model.predict(test_x)
    predictions = [round(value) for value in y_pred]
    
    accuracy = accuracy_score(test_y, predictions)
    print ("Accuracy: %.2f%%" % (accuracy * 100.0))

    return model



model_dict = {}
for col in vax_full.columns[1:]:
    global model_dict
    n = len(vax_full[col].unique())
    full_Xset = vax_full.drop(columns=[col,"VAERS_ID"])
    full_yset = vax_full[col]
    if col in ["VAX_TYPE","VAX_MANU","VAX_ROUTE"] :
        print(col, "KNN")
        model_dict[col] = knn_train(full_Xset, full_yset,n)
    else:
        print(col, "XGBoost")
        model_dict[col] = xgb_train(full_Xset, full_yset, 10)
    
    #display(full_Xset.head(3))

VAX_TYPE KNN
Accuracy: 51.89%
VAX_MANU KNN
Accuracy: 81.72%
VAX_DOSE_SERIES XGBoost
Accuracy: 63.04%
VAX_ROUTE KNN
Accuracy: 93.30%
VAX_SITE XGBoost
Accuracy: 53.72%


In [8]:
# Replace missing values
vax_fill = vax_coded[vax_coded.isnull().any(axis=1)]
print(vax_fill.shape)
display(vax_fill.head())
cc = 0

def fill(df_full, df_fill, df):
    global cc
    for index, row in df_fill.iterrows():
        for col in vax_fill.columns[1:]:
            cols = list(df.columns[1:])
            cols.remove(col)
            if np.isnan(row[col]):
                vax_fill.loc[index, col] = int(model_dict[col].predict(np.array(df_fill.loc[index,cols]).reshape(1, -1)))
                cc+=1

    vax_clean = pd.concat([vax_full, vax_fill])
    
    return vax_clean

vax_clean = fill(vax_full, vax_fill, vax_coded)

(27499, 6)


,VAERS_ID,VAX_TYPE,VAX_MANU,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE
4,617014,51,NaN,1,7,2
8,617017,19,5,NaN,1,2
9,617018,20,7,NaN,1,2
11,617020,26,11,NaN,2,6
12,617021,29,NaN,0,7,2


D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitte

D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitte

D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitte

...

D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitte

D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitte

D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitte

D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitte

D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitte

In [9]:
# Check of encoded NaNs
print(vax_clean.isna().sum())
print(cc)

VAERS_ID           0
VAX_TYPE           0
VAX_MANU           0
VAX_DOSE_SERIES    0
VAX_ROUTE          0
VAX_SITE           0
dtype: int64
27499


## VAERS

In [10]:
print(vaers_16.shape,vaers_17.shape)
vaers = pd.concat([vaers_16, vaers_17], ignore_index=True)
vaers.replace(["UN","UNK", "Unkown", "nan"], np.nan, inplace=True)
vaers.replace(["None known","No","No."], "None", inplace=True)
print("Raw merge: ", vaers.shape)


attr_vaers = ['VAERS_ID', 'AGE_YRS', 'SEX',
       'SYMPTOM_TEXT','ER_VISIT',
       'HOSPITAL', 'HOSPDAYS',  'DISABLE', 'RECOVD', 
        'NUMDAYS', 'LAB_DATA', 'V_ADMINBY', 'V_FUNDBY',
       'OTHER_MEDS', 'CUR_ILL', 'HISTORY', 'PRIOR_VAX', 
       'FORM_VERS', 'ALLERGIES']  
        #dropped 'BIRTH_DEFECT' cuz empty
        # dropped 'SPLTTYPE' cuz included in vax
vaers = vaers[attr_vaers]
print("Selected ft.: ", vaers.shape)

#Ensure VAERS_ID is unique
print ("\nVAERS_ID is unique.\n" if vaers.shape[0]==len(vaers["VAERS_ID"].unique()) else "\nVAERS_ID isn't unique.\n")

# Check Missing values
for col in vaers.columns:
    print(col, "has %d missing values.\n %s" % ((vaers[col].isna().sum()), \
                                                "{0:.0f}%".format((vaers[col].isna().sum())/vaers.shape[0] * 100)))

display(vaers.head(3))
print(vaers.shape)
#print(vaers.dtypes)
#print(vaers["FORM_VERS"].unique())



# PROBLEMS
# Age: weird floats
# [Lab_data, Symptom_text, Other_meds, History, Prior_Vax]: Needs NLP

(45705, 35) (38910, 35)
Raw merge:  (84615, 35)
Selected ft.:  (84615, 19)

VAERS_ID is unique.

VAERS_ID has 0 missing values.
 0%
AGE_YRS has 27164 missing values.
 32%
SEX has 0 missing values.
 0%
SYMPTOM_TEXT has 3 missing values.
 0%
ER_VISIT has 71846 missing values.
 85%
HOSPITAL has 82102 missing values.
 97%
HOSPDAYS has 82712 missing values.
 98%
DISABLE has 83568 missing values.
 99%
RECOVD has 4088 missing values.
 5%
NUMDAYS has 24543 missing values.
 29%
LAB_DATA has 65203 missing values.
 77%
V_ADMINBY has 13250 missing values.
 16%
V_FUNDBY has 34473 missing values.
 41%
OTHER_MEDS has 34280 missing values.
 41%
CUR_ILL has 27241 missing values.
 32%
HISTORY has 49134 missing values.
 58%
PRIOR_VAX has 82081 missing values.
 97%
FORM_VERS has 0 missing values.
 0%
ALLERGIES has 74668 missing values.
 88%


,VAERS_ID,AGE_YRS,SEX,SYMPTOM_TEXT,ER_VISIT,HOSPITAL,HOSPDAYS,DISABLE,RECOVD,NUMDAYS,LAB_DATA,V_ADMINBY,V_FUNDBY,OTHER_MEDS,CUR_ILL,HISTORY,PRIOR_VAX,FORM_VERS,ALLERGIES
0,617010,73.00,F,"Shortly after patient was vaccinated, she star...",Y,NaN,NaN,NaN,U,0.0,None,OTH,PUB,"Meloxican 7.5 mg, zolpidem 5 mg, atenolol 25mg...",None,"Allergies on file: penicillins, barbituates, s...",NaN,1,NaN
1,617011,65.00,M,Soreness at injection site. Soreness continui...,NaN,NaN,NaN,NaN,N,0.0,NaN,OTH,OTH,NaN,None,None,short term soreness~Influenza (Seasonal) (no b...,1,NaN
2,617012,0.68,F,Fever over 102. Fussiness. Loose stool. Lack o...,NaN,NaN,NaN,NaN,U,0.0,NaN,PVT,OTH,NaN,None,None,NaN,1,NaN


(84615, 19)


## Dealing Missing values

In [11]:
print("Dealt with columns:\n")

print("Fill Y, N/None:\n")
# Assume N(None) for unknown
for col in ["ER_VISIT", "HOSPITAL", "DISABLE"]:
    vaers[col] = vaers[col].replace(np.nan, "N")
    print(col, vaers[col].unique(),"\n")
for col in ["LAB_DATA","OTHER_MEDS","CUR_ILL","HISTORY","PRIOR_VAX","ALLERGIES" ]:
    vaers[col] = vaers[col].replace([np.nan,"N"], "None")
    vaers[col] = vaers[col].map(lambda x: 'None' if x.startswith(('no','No','NO')) else x)
    print(col, vaers[col].unique(), "\n")
# Assume recovered for unknown
vaers["RECOVD"] = vaers["RECOVD"].replace([np.nan, "U"], "Y")
print("RECOVD",vaers["RECOVD"].unique(),"\n")
print("Fill 0:")
vaers["HOSPDAYS"] = vaers["HOSPDAYS"].replace(np.nan, 0)
print("HOSPDAYS",vaers["HOSPDAYS"].unique())
print(vaers.shape)

Dealt with columns:

Fill Y, N/None:

ER_VISIT ['Y' 'N'] 

HOSPITAL ['N' 'Y'] 

DISABLE ['N' 'Y'] 

LAB_DATA ['None' 'MRI, Spinal tap, several blood tests.'
 "ULTRASOUNDS ON 08/17/1999 AND EXTENSIVE ULTRASOUNDS, CT SCANS, MRI'S EXAMINATIONS, AND HEALTH CARE PROVIDER CARE SINCE THAT TIME NOW AS CONTINUOUS.   6 MONTHS OF ANTICOAGULANT THERAPY AFTER DISCHARGE FROM HOSPITAL IN 1999. LAST VACCINATION OF SERIES OF 3 WAS APPX 3 WEEKS BEFORE PORTAL VEIN THROMBOSIS WAS DIAGNOSED BY ULTRASOUND AND CT SCANS.  THE RELEVANT CONNECTION BETWEEN THE SERIES OF 3 HEPATITIS B VACCINATIONS AND THE PORTAL VEIN THROMBOSIS IS NOT KNOWN BY PATIENT AS FAR AS FACTUAL AND POSSIBLE IN HAPPENING."
 ...
 'Returned to Doctor on Thursday 12/14/17 for reading of TB test and reported to RN.'
 'Xray of Left Shoulder12/04/2017' '2016-to date'] 

OTHER_MEDS ['Meloxican 7.5 mg, zolpidem 5 mg, atenolol 25mg, Flovent HFA 110mcg, ProAir HFA, Advair Diskus 250/50'
 'None' 'Flonase daily' ...
 'Escitalopram; multi vitamin; calc

In [12]:
# drop rows with more than 1 missing value
vaers = vaers[vaers.isnull().sum(axis=1) < 2]
vaers.shape

(57277, 19)

## NLP

In [13]:
"HHHH".isupper()

True

In [14]:
text = "".join(str(vaers["SYMPTOM_TEXT"].to_list())[:])
text = "".join((i+" ") for i in text.split(" ") if not i.isupper())
fq_words = ["patient","received","was","were", "vaccine","weeks",
           ]
for w in fq_words:
    text = text.replace(w,'')
    
print(text[:1500])

['Shortly after   vaccinated, she started to feel an itching, tingling feeling in her throat.  Fearing that it  an allergic reaction, called 911.  The  remained alert, talking and breathing normally until paramedics arrived, though she stated that she started to feel additional tingling in her arms and chest.  She  transported by ambulance to emergency department. No treatment administered before paramedics arrived.', 'Soreness at injection site.  Soreness continuing after 2  following injection.', 'Fever over 102. Fussiness. Loose stool. Lack of sleep.', 'Several incidents of intense vomiting and diarrhea between 9:00 and 7:00 accompanied by mild chllls. Weakness and mild dizziness in the morning. Another 3 hours of deep sleep required to re stabilize, followed by acute thirstiness.', 'Extremely swollen parotid glands, fever, headache, malaise, loss of appetite.', 'Swelling, redness at site- 3" 5" hard painful to touch, pain in shoulder and neck like a pinched nerve which hasn\'t gone

In [27]:
# Specifying Parameters
language = "en"
max_ngram_size = 10
deduplication_thresold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 20

custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(text)

for kw in keywords:
    print(kw)

('dose of my immunization', -17977.822931839703)
('told that the swelling', -6143.826484464288)
('developed on my neck', -5521.60270065494)
('muscle in the upper', -3834.106226951167)
('doctors and so called', -3189.6585250250564)
('immunizations on the day', -2282.626081255313)
('hives in a female', -2214.26120702562)
('severe and also developed', -974.6572551957514)
('state that if felt', -795.9809098770911)
('called r and doctor', -672.6703093444988)
('lasted into the day', -592.3732035426895)
('painful after the shot', -575.0468052700814)
('developed and became severe', -465.76925375375424)
('recovered from the headache', -452.61834115402456)
('physician right away due', -451.67203927497553)
('office for further treatment', -401.82319587420017)
('office and no treatment', -386.08465483255014)
('thigh where the injection', -284.85454181657303)
('strength to the left', -253.0706069851301)
('day did not feel', -242.11248054026356)
('week and the physician', -233.68874525024557)
('phar

In [22]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [23]:
fqwords = ["patient","received","she","he","was","were", "vaccine", "in", "weeks"]
s=set(nltk.corpus.stopwords.words('english'))
word_tokens = nltk.tokenize.word_tokenize(text)
text = [w for w in word_tokens if not w.lower() in s]
print(text[:1500])

['[', "'hrl", 'pen', 'w', 'vccne', ',', 'feel', 'n', 'chng', ',', 'nglng', 'feelng', 'n', 'hr', '.', 'erng', 'h', 'w', 'n', 'ergc', 'cn', ',', 'ce', '911.', 'pen', 'e', 'ler', ',', 'lkng', 'bhng', 'r', 'prec', 'rr', ',', 'hugh', 'e', 'h', 'feel', 'nl', 'nglng', 'n', 'r', 'c.', 'w', 'rnpre', 'bulnce', 'eergenc', 'epren', '.', 'en', 'ne', 'f', 'prec', 'rr', '.', "'", ',', "'ne", 'njecn', 'e.', 'ne', 'cnnung', '2', 'ek', 'fwng', 'njecn', '.', "'", ',', "'er", 'r', '102.', 'une', '.', 'e', 'l.', 'ck', 'leep', '.', "'", ',', "'erl", 'ncen', 'nene', 'vng', 'rr', '9:00', '7:00', 'ccpne', 'l', 'chl', '.', 'ekne', 'l', 'zzne', 'n', 'rnng', '.', 'r', '3', 'h', 'eep', 'leep', 'qu', 'blze', ',', 'f', 'cue', 'hrne', '.', "'", ',', "'xel", 'n', 'pr', 'gl', ',', 'fer', ',', 'c', ',', 'le', ',', 'l', 'ppee', '.', "'", ',', "'ng", ',', 'ne', 'e-', '3', "''", '5', "''", 'hr', 'pful', 'uch', ',', 'p', 'n', 'er', 'neck', 'lke', 'pnc', 'ner', 'whch', '\\', "'", 'gne', 'w.', "'", ',', "'njecn", 'e', 'cn', '

### Fill with

In [65]:
# Get full subset
vaers_full = vaers.dropna()
print("full subset: ", vaers_full.shape)

full subset:  (5173, 20)


In [354]:
# Check Missing values
for col in vaers.columns:
    print(col, "has %d missing values.\n %s" % ((vaers[col].isna().sum()), \
                                                "{0:.0f}%".format((vaers[col].isna().sum())/vaers.shape[0] * 100)))

VAERS_ID has 0 missing values.
 0%
AGE_YRS has 6864 missing values.
 12%
SEX has 0 missing values.
 0%
SYMPTOM_TEXT has 1 missing values.
 0%
ER_VISIT has 0 missing values.
 0%
HOSPITAL has 0 missing values.
 0%
HOSPDAYS has 0 missing values.
 0%
DISABLE has 0 missing values.
 0%
RECOVD has 0 missing values.
 0%
NUMDAYS has 3482 missing values.
 6%
LAB_DATA has 0 missing values.
 0%
V_ADMINBY has 2875 missing values.
 5%
V_FUNDBY has 15852 missing values.
 28%
OTHER_MEDS has 0 missing values.
 0%
CUR_ILL has 0 missing values.
 0%
HISTORY has 0 missing values.
 0%
PRIOR_VAX has 0 missing values.
 0%
FORM_VERS has 0 missing values.
 0%
ALLERGIES has 0 missing values.
 0%


## Combined Input dataset

In [371]:
df_input = vaers.merge(vax_clean, how="inner", left_index=True, right_index=True)
df_input.drop(columns="VAERS_ID_y", inplace=True)
df.rename(columns={"VAERS_ID_x": "VAERS_ID"})
print(df_input.shape)
df_input.head(3)

(32414, 24)


,VAERS_ID_x,AGE_YRS,SEX,SYMPTOM_TEXT,ER_VISIT,HOSPITAL,HOSPDAYS,DISABLE,RECOVD,NUMDAYS,...,CUR_ILL,HISTORY,PRIOR_VAX,FORM_VERS,ALLERGIES,VAX_TYPE,VAX_MANU,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE
0,617010,73.0,F,"Shortly after patient was vaccinated, she star...",Y,N,0.0,N,Y,0.0,...,None,"Allergies on file: penicillins, barbituates, s...",None,1,None,19,19,0,1,2
1,617011,65.0,M,Soreness at injection site. Soreness continui...,N,N,0.0,N,N,0.0,...,None,None,short term soreness~Influenza (Seasonal) (no b...,1,None,20,7,0,1,8
4,617014,5.0,F,"Extremely swollen parotid glands, fever, heada...",N,N,0.0,N,Y,8.0,...,None,None,None,1,None,51,12,1,7,2
